### Conda Env

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:48
🔁 Restarting kernel...


In [ ]:
!conda --version

!conda install geopandas
!conda install rioxarray
!conda install geojson
!conda install rasterstats

conda 4.9.2
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ 

In [ ]:
# https://stackoverflow.com/questions/49154899/resolvepackagenotfound-create-env-using-conda-and-yml-file-on-macos

#!pip install rioxarray
#!pip install xarray-spatial
!conda install cffi==1.15.0

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cffi==1.15.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cffi-1.15.0                |   py37h036bc23_0         225 KB  conda-forge
 

In [ ]:
!conda list

# packages in environment at /usr/local:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       1_gnu    conda-forge
affine                    2.3.0                    pypi_0    pypi
attrs                     21.4.0             pyhd8ed1ab_0    conda-forge
bokeh                     2.4.2            py37h89c1867_0    conda-forge
boost-cpp                 1.74.0               hc6e9bd1_3    conda-forge
branca                    0.4.2              pyhd8ed1ab_0    conda-forge
brotlipy                  0.7.0           py37h5e8e339_1001    conda-forge
bzip2                     1.0.8                h7f98852_4    conda-forge
c-ares                    1.17.1               h7f98852_1    conda-forge
ca-certificates           2021.10.8            ha878542_0    conda-forge
cairo                     1.16.0            h6cf1ce9_1008    conda-forge
certifi            

## Load Packages

In [1]:
# Link to Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install geopandas
!pip install rioxarray
!pip install geojson
!pip install rasterstats

import os
from glob import glob
import numpy as np
import pandas as pd
import geojson
import shapely as shp
from shapely.geometry import Polygon, box
import geopandas as gpd
import xarray as xr
import rioxarray as rioxr
from rasterstats import zonal_stats
import warnings
warnings.filterwarnings("ignore")

     |████████████████████████████████| 1.0 MB 5.3 MB/s 
     |████████████████████████████████| 6.3 MB 44.6 MB/s 
     |████████████████████████████████| 15.4 MB 34.8 MB/s 
     |████████████████████████████████| 47 kB 2.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 19.3 MB 1.3 MB/s 
  Created wheel for rioxarray: filename=rioxarray-0.9.1-py3-none-any.whl size=54611 sha256=da7a87d56a0c04cb59c2deb57673b61844537de5e39adf14046da8996e69a0a4
  Stored in directory: /root/.cache/pip/wheels/07/da/9e/1cc57b2e7a29a206893db83e984a341e2e94378263e0798229
Successfully built rioxarray
     |████████████████████████████████| 130 kB 5.2 MB/s 


## Load All Parcels

In [ ]:
# Load all parcels
os.chdir('/content/drive/MyDrive/THESIS_AQUAPONDS/ROI/Aquaculture_Asia_Coast_2019/Coastline_parcels_polygon')
path_parcels = glob(os.path.join(os.getcwd(), 'parcels.geojson'))[0]
parcels = gpd.read_file(path_parcels)
parcels.rename(columns={'id':'parcelID'}, inplace=True)
parcels.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   fid         261 non-null    int64   
 1   TARGET_FID  261 non-null    int64   
 2   GID_0       261 non-null    object  
 3   NAME_0      261 non-null    object  
 4   Shape_Area  261 non-null    float64 
 5   geometry    261 non-null    geometry
dtypes: float64(1), geometry(1), int64(2), object(2)
memory usage: 12.4+ KB


In [ ]:
parcelIDs = list(parcels['TARGET_FID'])
len(parcelIDs)

261

In [ ]:
parcelIDs[0]

215

In [ ]:
#parcels.to_file('parcels.geojson', driver='GeoJSON')
with open('parcels.geojson') as f:
  parcels_geojson = geojson.load(f)

## Tile Ponds by Parcels


### based on spatial insection (discarded, since *parcel* attributes are less accurate than ponds' attributes)

In [ ]:
# Load All Ponds
path_ponds = '/content/drive/My Drive/THESIS_AQUAPONDS/ROI/Aquaculture_Asia_Coast_2019/Aquaculture_ponds_Asian_coast_2019/Asia_aquaculture_ponds_2019_plusGADM.gpkg'
ponds_all = gpd.read_file(path_ponds)
ponds_all.head()

,id,parcel_number,distance,elevation,GID_0,NAME_0,GID_1,NAME_1,ENGTYPE_1,GID_2,NAME_2,geometry
0,88082,5302,7845.0,20,VNM,Vietnam,VNM.49_1,Qu?ng Ninh,Province,VNM.49.10_1,Móng Cái,"MULTIPOLYGON (((107.83395 21.60565, 107.83404 ..."
1,88086,5302,7814.0,17,VNM,Vietnam,VNM.49_1,Qu?ng Ninh,Province,VNM.49.10_1,Móng Cái,"MULTIPOLYGON (((107.83440 21.60511, 107.83440 ..."
2,88088,5302,7326.0,21,VNM,Vietnam,VNM.49_1,Qu?ng Ninh,Province,VNM.49.10_1,Móng Cái,"MULTIPOLYGON (((107.83467 21.60089, 107.83484 ..."
3,88089,5302,7837.0,13,VNM,Vietnam,VNM.49_1,Qu?ng Ninh,Province,VNM.49.10_1,Móng Cái,"MULTIPOLYGON (((107.83484 21.60556, 107.83502 ..."
4,88095,5302,7764.0,18,VNM,Vietnam,VNM.49_1,Qu?ng Ninh,Province,VNM.49.10_1,Móng Cái,"MULTIPOLYGON (((107.83538 21.60466, 107.83538 ..."


In [ ]:
os.chdir('/content/drive/My Drive/THESIS_AQUAPONDS/ROI/Aquaculture_Asia_Coast_2019/Aquaculture_ponds_Asian_coast_2019/ponds_by_parcel/')
os.getcwd()

# Build Generator Function to tile ponds
# Reference: https://stackoverflow.com/questions/37156574/why-does-a-generator-expression-need-a-lot-of-memory/37156765:
def tile_ponds_by_parcel(n):
  parcelID = 95
  while parcelID < n:
    ponds_sub = ponds_all[ponds_all.geometry.within(parcels[parcels['parcelID']==parcelID].geometry.unary_union)]
    if ponds_sub.empty:
      del ponds_sub
    else:
      yield ponds_sub.to_file('ponds_pa_'+str(parcelID)+'.geojson', driver='GeoJSON')
    parcelID += 1

gen = tile_ponds_by_parcel(len(parcels['parcelID']))

In [ ]:
# Exhaust Generator
# Reference: https://stackoverflow.com/questions/47456631/simpler-way-to-run-a-generator-function-without-caring-about-items
from collections import deque

def exhaust(generator):
    deque(generator, maxlen=0)

exhaust(gen)

### based on ponds' attribute table

In [ ]:
# Load All Ponds
path_ponds = '/content/drive/My Drive/THESIS_AQUAPONDS/ROI/Aquaculture_Asia_Coast_2019/Aquaculture_ponds_Asian_coast_2019/Asia_aquaculture_ponds_2019_plusGADM.gpkg'
ponds_all = gpd.read_file(path_ponds)
ponds_all.head()

,id,parcel_number,distance,elevation,GID_0,NAME_0,GID_1,NAME_1,ENGTYPE_1,GID_2,NAME_2,geometry
0,88082,5302,7845.0,20,VNM,Vietnam,VNM.49_1,Qu?ng Ninh,Province,VNM.49.10_1,Móng Cái,"MULTIPOLYGON (((107.83395 21.60565, 107.83404 ..."
1,88086,5302,7814.0,17,VNM,Vietnam,VNM.49_1,Qu?ng Ninh,Province,VNM.49.10_1,Móng Cái,"MULTIPOLYGON (((107.83440 21.60511, 107.83440 ..."
2,88088,5302,7326.0,21,VNM,Vietnam,VNM.49_1,Qu?ng Ninh,Province,VNM.49.10_1,Móng Cái,"MULTIPOLYGON (((107.83467 21.60089, 107.83484 ..."
3,88089,5302,7837.0,13,VNM,Vietnam,VNM.49_1,Qu?ng Ninh,Province,VNM.49.10_1,Móng Cái,"MULTIPOLYGON (((107.83484 21.60556, 107.83502 ..."
4,88095,5302,7764.0,18,VNM,Vietnam,VNM.49_1,Qu?ng Ninh,Province,VNM.49.10_1,Móng Cái,"MULTIPOLYGON (((107.83538 21.60466, 107.83538 ..."


In [ ]:
parcelNos = list(set(list(ponds_all['parcel_number'])))
len(parcelNos)

388

In [ ]:
os.chdir('/content/drive/My Drive/THESIS_AQUAPONDS/ROI/Aquaculture_Asia_Coast_2019/Aquaculture_ponds_Asian_coast_2019/ponds_by_parcel_byAttribute_388p/')

# Build Generator Function to tile ponds
# Reference: https://stackoverflow.com/questions/37156574/why-does-a-generator-expression-need-a-lot-of-memory/37156765:
def tile_ponds_by_parcel(n):
  parcelNos = list(set(list(ponds_all['parcel_number'])))
  i = 0
  while i < n:
    ponds_sub = ponds_all[ponds_all['parcel_number']==parcelNos[i]]
    yield ponds_sub.to_file('ponds_pa_'+str(parcelNos[i])+'.geojson', driver='GeoJSON')
    i += 1

gen = tile_ponds_by_parcel(len(parcelNos))

In [ ]:
# Exhaust Generator
# Reference: https://stackoverflow.com/questions/47456631/simpler-way-to-run-a-generator-function-without-caring-about-items
from collections import deque

def exhaust(generator):
    deque(generator, maxlen=0)

exhaust(gen)

### Merge Ponds that have same Parcel ID

In [ ]:
os.chdir('/content/drive/My Drive/THESIS_AQUAPONDS/ROI/Aquaculture_Asia_Coast_2019/Aquaculture_ponds_Asian_coast_2019/ponds_by_parcel_byAttribute_388p/')
files = glob(os.path.join(os.getcwd(), '*'+str(parcelIDs[0])+'*'))
dfs = [gpd.read_file(i) for i in files]
df_merged = pd.concat([df for df in dfs])
df_merged.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   id             37 non-null     int64   
 1   parcel_number  37 non-null     int64   
 2   distance       37 non-null     float64 
 3   elevation      37 non-null     int64   
 4   GID_0          37 non-null     object  
 5   NAME_0         37 non-null     object  
 6   GID_1          37 non-null     object  
 7   NAME_1         37 non-null     object  
 8   ENGTYPE_1      37 non-null     object  
 9   GID_2          37 non-null     object  
 10  NAME_2         37 non-null     object  
 11  geometry       37 non-null     geometry
dtypes: float64(1), geometry(1), int64(3), object(7)
memory usage: 3.6+ KB


In [ ]:
os.chdir('/content/drive/My Drive/THESIS_AQUAPONDS/ROI/Aquaculture_Asia_Coast_2019/Aquaculture_ponds_Asian_coast_2019/ponds_by_parcel_byAttribute_261p/')

def merge_ponds_same_parcel(n):
  input_path = '/content/drive/My Drive/THESIS_AQUAPONDS/ROI/Aquaculture_Asia_Coast_2019/Aquaculture_ponds_Asian_coast_2019/ponds_by_parcel_byAttribute_388p/'
  i = 0
  while i < n:
    files = glob(os.path.join(input_path, '*'+str(parcelIDs[i])+'*'))

    if len(files)==0:
      i += 1
    else:
      dfs = [gpd.read_file(j) for j in files]
      df_merged = pd.concat([df for df in dfs])
      yield df_merged.to_file('ponds_pa_'+str(parcelIDs[i])+'.geojson', driver='GeoJSON')
      i += 1

gen = merge_ponds_same_parcel(len(parcelIDs))

In [ ]:
# Exhaust Generator
# Reference: https://stackoverflow.com/questions/47456631/simpler-way-to-run-a-generator-function-without-caring-about-items
from collections import deque

def exhaust(generator):
    deque(generator, maxlen=0)

exhaust(gen)

## Zonal Stats Processing Flow

### Global Setting & Functions

In [80]:
# Pair up Index and Years
keys = [str(i) for i in range(36)]
#years = pd.date_range(start='1984', end='2020', freq='Y')
years = [i for i in range(1984,2020)]
pair = dict([[i,j] for i,j in zip(keys, years)])
print(pair)

{'0': 1984, '1': 1985, '2': 1986, '3': 1987, '4': 1988, '5': 1989, '6': 1990, '7': 1991, '8': 1992, '9': 1993, '10': 1994, '11': 1995, '12': 1996, '13': 1997, '14': 1998, '15': 1999, '16': 2000, '17': 2001, '18': 2002, '19': 2003, '20': 2004, '21': 2005, '22': 2006, '23': 2007, '24': 2008, '25': 2009, '26': 2010, '27': 2011, '28': 2012, '29': 2013, '30': 2014, '31': 2015, '32': 2016, '33': 2017, '34': 2018, '35': 2019}


In [81]:
def tidy_zonstats_df(zonstats_dict, gdf, year):
  # Tidy dataframe of zonal statistics
  df_stats = pd.DataFrame(zonstats_dict)

  if 0 not in df_stats:
    df_stats[0] = np.nan
  if 1 not in df_stats:
    df_stats[1] = np.nan
  if 2 not in df_stats:
    df_stats[2] = np.nan

  # Convert values to proper data type
  df_stats = df_stats.convert_dtypes()
  # Rename columns
  colname_c0 = 'c0_'+str(year)
  colname_c1 = 'c1_'+str(year)
  colname_c2 = 'c2_'+str(year)
  df_stats.rename(columns={0:colname_c0, 1:colname_c1, 2:colname_c2}, inplace=True)
  # Add 'pondID' as new column
  df_stats = df_stats.assign(pondID = np.array(gdf['pondID']))
  df_stats[[colname_c0, colname_c1, colname_c2]] = df_stats[[colname_c0, colname_c1, colname_c2]].fillna(0)
  return df_stats


In [82]:
def zonal_statistics(year, gdf, xds, affine):

  # Subset raster to one year
  xda_1Y = xds.sel(time=year).to_array()
  # Convert DataArray to NumpyArray
  array = xda_1Y.values[0]

  # 1st Run Zonal statistics
  zonstats_1st = zonal_stats(gdf, array, transform=affine, categorical=True, all_touched=False) 

  # Tidy dataframe of zonal statistics
  df_stats = tidy_zonstats_df(zonstats_1st, gdf, year)

  # Store needed Stats from Run 1 and Fetch Ponds for Run 2
  colname_c0 = 'c0_'+str(year)
  colname_c1 = 'c1_'+str(year)
  colname_c2 = 'c2_'+str(year)
  stats_run1 = df_stats[(df_stats[colname_c0]!=0) | (df_stats[colname_c1]!=0) | (df_stats[colname_c2]!=0)]
  ponds_run2 = gdf.merge(df_stats[(df_stats[colname_c0]==0) & (df_stats[colname_c1]==0) & (df_stats[colname_c2]==0)], how='right', on='pondID')

  if ponds_run2.empty:

    zonstats = stats_run1

  else:
    
    # 2nd Run Zonal Stats
    zonstats_2nd = zonal_stats(ponds_run2, array, transform=affine, categorical=True, all_touched=True)

    # Tidy dataframe of zonal statistics
    stats_run2 = tidy_zonstats_df(zonstats_2nd, ponds_run2, year) 

    # Join Outputs
    zonstats = pd.concat([stats_run1, stats_run2])

  return zonstats

### Input

In [83]:
# Directory "Images"
path_img = '/content/drive/My Drive/myExportImage/test_data_for_marco/Img_waterMsk/'
Images = glob(os.path.join(path_img, '*.tif'))

# Directory "Ponds"
path_ponds = '/content/drive/My Drive/myExportImage/test_data_for_marco/ponds_by_parcel/'

In [84]:
# Parcel IDs
parcelIDs = sorted(list(set([i.split('/')[-1].split('.')[0].split('-')[1] for i in Images])))
print('Parcel IDs:', parcelIDs)

Parcel IDs: ['215', '482', '517', '567', '601', '640', '667', '689', '692']


### Iteration

* **Big Loop (over parcelIDs) Starts!**

In [85]:
# ID of Parcel_OO
parcelID = parcelIDs[4]
print('Parcel ID: ', parcelID)

Parcel ID:  601


In [86]:
# Ponds of Parcel_OO
pondItem = glob(os.path.join(path_ponds, '*'+parcelID+'*'))
ponds = gpd.read_file(pondItem[0])
ponds.rename(columns={'id':'pondID'}, inplace=True)
print('Ponds of Parcel: ', pondItem)

# Images of Parcel_OO
imageItems = glob(os.path.join(path_img, '*'+parcelID+'*.tif'))
print('Images of Parcel:', imageItems)

Ponds of Parcel:  ['/content/drive/My Drive/myExportImage/test_data_for_marco/ponds_by_parcel/ponds_pa_601.geojson']
Images of Parcel: ['/content/drive/My Drive/myExportImage/test_data_for_marco/Img_waterMsk/p-601.tif']


In [87]:
# Tidy Images of Parcel_OO

elements = [rioxr.open_rasterio(i) for i in imageItems]
xdas = [i.rename({'band':'time'}) for i in elements]
xdss = [i.to_dataset(name='watermask') for i in xdas]
xdss = [i.reindex(y = list(reversed(i.y))) for i in xdss]

# Get the index of loaded image
imgID = [i.split('_')[0] for i in xdas[0].attrs['long_name']]
# Get years based on index
t = [pair[i] for i in imgID]

for i in range(len(xdss)):
  xdss[i].coords['time'] = np.array(t)
#print(xdss)

In [88]:
# "Affine Transformations" of Images of Parcel_OO
transform = [i.rio.transform() for i in xdss]
we_pr, rotation, ns_pr = [i[0] for i in transform], [i[1] for i in transform], [i[4] for i in transform]
xmin, xmax, ymin, ymax = [i.x.min().values for i in xdss], [i.x.max().values for i in xdss], [i.y.min().values for i in xdss], [i.y.max().values for i in xdss]
affines = [[a,b,c,d,e,f] for a,b,c,d,e,f in zip(xmin, we_pr, rotation, ymax, rotation, ns_pr)]
#affines

In [89]:
# Bounding Boxes of Images of Parcel_OO
bbox = [box(minx, miny, maxx, maxy) for minx, miny, maxx, maxy in zip(xmin, ymin, xmax, ymax)]

# Tiled Ponds of Parcel_OO
ponds_sub = [gpd.GeoDataFrame(ponds[ponds.geometry.within(i)]) for i in bbox]

In [90]:
# Remove Items from "tiled ponds", "tidy images" and "affines" where the ponds subset is empty.
index_to_keep = [k for k,i in enumerate(ponds_sub) if i.empty!=True]

# Filtered Ponds Tiles of Parcel_OO
ponds_input = [i for k,i in enumerate(ponds_sub) if k in index_to_keep]
# Filtered Tidy Images of Parcel_OO
xdss_input = [i for k,i in enumerate(xdss) if k in index_to_keep]
# Filtered Affines of Parcel_OO
affines_input = [i for k,i in enumerate(affines) if k in index_to_keep]

In [91]:
len(affines_input)

1

* **Small Loop (over ponds tiles of Parcel_OO) Starts!**

In [92]:
years = t[-2:]
zonstats_listNested = [[zonal_statistics(year=i, gdf=ponds_input[j].iloc[:20], xds=xdss_input[j], affine=affines_input[j]) for i in years] for j in range(len(ponds_input))]
zonstats_listFlat = [pd.concat([df.set_index('pondID') for df in j], axis=1) for j in zonstats_listNested]
ponds_zonstats_list = [i.iloc[:20].merge(j, how='outer', on='pondID') for i,j in zip(ponds_input, zonstats_listFlat)]

In [93]:
zonstats_listNested

[[    c2_2018  c0_2018  c1_2018  pondID
  0         2        0        0   27259
  3         2        0        0   27262
  8         1        0        0   27270
  11        1        0        0   27276
  12        1        0        0   27277
  13        1        0        0   27280
  18        1        0        0   27298
  19        1        0        0   27299
  0         1        0        0   27260
  1         2        0        0   27261
  2         4        0        0   27263
  3         4        0        0   27264
  4         2        0        0   27265
  5         2        0        0   27266
  6         2        0        0   27271
  7         2        0        0   27275
  8         2        0        0   27288
  9         2        0        0   27289
  10        1        0        0   27291
  11        1        0        0   27292,     c2_2019  c0_2019  c1_2019  pondID
  0         2        0        0   27259
  3         2        0        0   27262
  8         1        0        0   27270
 

### Output

In [94]:
# df zonStats of Parcel_OO
ponds_zonstats = pd.concat([df for df in ponds_zonstats_list])
#ponds_zonstats.info()
ponds_zonstats

,pondID,parcel_number,distance,elevation,GID_0,NAME_0,GID_1,NAME_1,ENGTYPE_1,GID_2,NAME_2,geometry,c2_2018,c0_2018,c1_2018,c2_2019,c0_2019,c1_2019
0,27259,601,220.0,9,IDN,Indonesia,IDN.4_1,Banten,Province,IDN.4.5_1,Pandeglang,"MULTIPOLYGON (((105.10504 -6.61088, 105.10558 ...",2,0,0,2,0,0
1,27260,601,242.0,7,IDN,Indonesia,IDN.4_1,Banten,Province,IDN.4.5_1,Pandeglang,"MULTIPOLYGON (((105.10576 -6.61079, 105.10576 ...",1,0,0,1,0,0
2,27261,601,247.0,8,IDN,Indonesia,IDN.4_1,Banten,Province,IDN.4.5_1,Pandeglang,"MULTIPOLYGON (((105.10603 -6.61043, 105.10621 ...",2,0,0,2,0,0
3,27262,601,244.0,7,IDN,Indonesia,IDN.4_1,Banten,Province,IDN.4.5_1,Pandeglang,"MULTIPOLYGON (((105.10630 -6.60980, 105.10648 ...",2,0,0,2,0,0
4,27263,601,63.0,9,IDN,Indonesia,IDN.4_1,Banten,Province,IDN.4.5_1,Pandeglang,"MULTIPOLYGON (((105.10648 -6.60774, 105.10657 ...",4,0,0,4,0,0
5,27264,601,0.0,17,IDN,Indonesia,IDN.4_1,Banten,Province,IDN.4.5_1,Pandeglang,"MULTIPOLYGON (((105.15634 -6.58178, 105.15634 ...",4,0,0,4,0,0
6,27265,601,6.0,17,IDN,Indonesia,IDN.4_1,Banten,Province,IDN.4.5_1,Pandeglang,"MULTIPOLYGON (((105.19164 -6.66622, 105.19164 ...",2,0,0,2,0,0
7,27266,601,4.0,8,IDN,Indonesia,IDN.4_1,Banten,Province,IDN.4.5_1,Pandeglang,"MULTIPOLYGON (((105.21176 -6.76261, 105.21176 ...",2,0,0,2,0,0
8,27270,601,738.0,12,IDN,Indonesia,IDN.4_1,Banten,Province,IDN.4.5_1,Pandeglang,"MULTIPOLYGON (((105.22147 -6.62795, 105.22147 ...",1,0,0,1,0,0
9,27271,601,734.0,12,IDN,Indonesia,IDN.4_1,Banten,Province,IDN.4.5_1,Pandeglang,"MULTIPOLYGON (((105.22156 -6.62804, 105.22156 ...",2,0,0,2,0,0


In [95]:
outputPath_zonstatsDF = '/content/drive/My Drive/THESIS_AQUAPONDS/gdf_zonstats'
ponds_zonstats.to_file(os.path.join(outputPath_zonstatsDF, 'zonstats_p_'+str(parcelID)+'.geojson'), driver='GeoJSON')

In [68]:
#test = gpd.read_file(os.path.join(outputPath_zonstatsDF, 'zonstats_p_151.geojson'))
#test

## TEST: Opened Zonal Stats Function

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [76]:
ponds_test = ponds_input[0].iloc[0:20]
gdf=ponds_input[j].iloc[:20]
#ponds_test

In [77]:
xda_test = xdss_input[0].sel(time=2019).to_array()
# Convert DataArray to numpy array
# https://stackoverflow.com/questions/55132416/converting-dataarray-into-numpy-array
array = xda_test.values[0]
array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [78]:
zonstats_1st = zonal_stats(ponds_test, array, transform=affines_input[0], categorical=True, all_touched=False) 
print(zonstats_1st)

[{2: 2}, {}, {}, {2: 2}, {}, {}, {}, {}, {2: 1}, {}, {}, {2: 1}, {2: 1}, {2: 1}, {}, {}, {}, {}, {2: 1}, {2: 1}]


In [79]:
# Tidy dataframe of zonal statistics
  df_stats = tidy_zonstats_df(zonstats_1st, gdf, year)

  # Store needed Stats from Run 1 and Fetch Ponds for Run 2
  colname_c0 = 'c0_'+str(year)
  colname_c1 = 'c1_'+str(year)
  colname_c2 = 'c2_'+str(year)
  stats_run1 = df_stats[(df_stats[colname_c0]!=0) | (df_stats[colname_c1]!=0) | (df_stats[colname_c2]!=2)]
  ponds_run2 = gdf.merge(df_stats[(df_stats[colname_c0]==0) & (df_stats[colname_c1]==0) & (df_stats[colname_c2]==0)], how='right', on='pondID')


df_stats = tidy_zonstats_df(zonstats_1st, ponds_test, t[-1])
print(df_stats)
stats_run1 = df_stats[(df_stats['c0_2019']!=0) | (df_stats['c1_2019']!=0) | (df_stats['c2_2019']!=0)]
ponds_run2 = ponds_test.merge(df_stats[(df_stats['c0_2019']==0) & (df_stats['c1_2019']==0) & (df_stats['c2_2019']==0)], how='right', on='pondID')
print(stats_run1.shape, ponds_run2.shape)
stats_run1.head()

    c2_2019  c0_2019  c1_2019  pondID
0         2        0        0   27259
1         0        0        0   27260
2         0        0        0   27261
3         2        0        0   27262
4         0        0        0   27263
5         0        0        0   27264
6         0        0        0   27265
7         0        0        0   27266
8         1        0        0   27270
9         0        0        0   27271
10        0        0        0   27275
11        1        0        0   27276
12        1        0        0   27277
13        1        0        0   27280
14        0        0        0   27288
15        0        0        0   27289
16        0        0        0   27291
17        0        0        0   27292
18        1        0        0   27298
19        1        0        0   27299
(8, 4) (12, 15)


,c2_2019,c0_2019,c1_2019,pondID
0,2,0,0,27259
3,2,0,0,27262
8,1,0,0,27270
11,1,0,0,27276
12,1,0,0,27277


In [50]:
zonstats_2nd = zonal_stats(ponds_run2, array, transform=affines_input[0], categorical=True, all_touched=True) 
print(zonstats_2nd)

[{2: 1}, {2: 2}, {2: 4}, {2: 4}, {2: 2}, {2: 2}, {2: 2}, {2: 2}, {2: 2}, {2: 2}, {2: 1}, {2: 1}]


In [51]:
colnames = ['c0_2019', 'c1_2019', 'c2_2019']

df_test = pd.DataFrame(zonstats_2nd)
print(df_test.shape)

if 0 not in df_test:
  df_test['0'] = np.nan
if 1 not in df_test:
  df_test['1'] = np.nan
if 2 not in df_test:
  df_test['2'] = np.nan
df_test

(12, 1)


,2,0,1
0,1,NaN,NaN
1,2,NaN,NaN
2,4,NaN,NaN
3,4,NaN,NaN
4,2,NaN,NaN
5,2,NaN,NaN
6,2,NaN,NaN
7,2,NaN,NaN
8,2,NaN,NaN
9,2,NaN,NaN


In [54]:
df_stats = pd.DataFrame(zonstats_2nd)
if 0 not in df_stats:
  df_stats[0] = np.nan
if 1 not in df_test:
  df_stats[1] = np.nan
if 2 not in df_test:
  df_test['2'] = np.nan
df_stats = df_stats.convert_dtypes()
# Rename columns
df_stats.rename(columns={0:'c0_2019', 1:'c1_2019', 2:'c2_2019'}, inplace=True)
# Add 'pondID' as new column
df_stats = df_stats.assign(pondID = np.array(ponds_run2['pondID']))
df_stats[['c0_2019', 'c1_2019', 'c2_2019']] = df_stats[['c0_2019', 'c1_2019', 'c2_2019']].fillna(0)
df_stats

stats_run2 = df_stats
stats_run2 = tidy_zonstats_df(zonstats_2nd, ponds_run2, t[-1])
stats_run2#.head()

,c2_2019,c0_2019,c1_2019,pondID
0,1,0,0,27260
1,2,0,0,27261
2,4,0,0,27263
3,4,0,0,27264
4,2,0,0,27265
5,2,0,0,27266
6,2,0,0,27271
7,2,0,0,27275
8,2,0,0,27288
9,2,0,0,27289


In [55]:
zonstats = pd.concat([stats_run1, stats_run2])
zonstats#.head()

,c2_2019,c0_2019,c1_2019,pondID
0,2,0,0,27259
3,2,0,0,27262
8,1,0,0,27270
11,1,0,0,27276
12,1,0,0,27277
13,1,0,0,27280
18,1,0,0,27298
19,1,0,0,27299
0,1,0,0,27260
1,2,0,0,27261


In [69]:
ponds_zonstats = ponds_test.merge(zonstats, how='outer', on='pondID')
ponds_zonstats

,pondID,parcel_number,distance,elevation,GID_0,NAME_0,GID_1,NAME_1,ENGTYPE_1,GID_2,NAME_2,geometry,c2_2019,c0_2019,c1_2019
0,27259,601,220.0,9,IDN,Indonesia,IDN.4_1,Banten,Province,IDN.4.5_1,Pandeglang,"MULTIPOLYGON (((105.10504 -6.61088, 105.10558 ...",2,0,0
1,27260,601,242.0,7,IDN,Indonesia,IDN.4_1,Banten,Province,IDN.4.5_1,Pandeglang,"MULTIPOLYGON (((105.10576 -6.61079, 105.10576 ...",1,0,0
2,27261,601,247.0,8,IDN,Indonesia,IDN.4_1,Banten,Province,IDN.4.5_1,Pandeglang,"MULTIPOLYGON (((105.10603 -6.61043, 105.10621 ...",2,0,0
3,27262,601,244.0,7,IDN,Indonesia,IDN.4_1,Banten,Province,IDN.4.5_1,Pandeglang,"MULTIPOLYGON (((105.10630 -6.60980, 105.10648 ...",2,0,0
4,27263,601,63.0,9,IDN,Indonesia,IDN.4_1,Banten,Province,IDN.4.5_1,Pandeglang,"MULTIPOLYGON (((105.10648 -6.60774, 105.10657 ...",4,0,0
5,27264,601,0.0,17,IDN,Indonesia,IDN.4_1,Banten,Province,IDN.4.5_1,Pandeglang,"MULTIPOLYGON (((105.15634 -6.58178, 105.15634 ...",4,0,0
6,27265,601,6.0,17,IDN,Indonesia,IDN.4_1,Banten,Province,IDN.4.5_1,Pandeglang,"MULTIPOLYGON (((105.19164 -6.66622, 105.19164 ...",2,0,0
7,27266,601,4.0,8,IDN,Indonesia,IDN.4_1,Banten,Province,IDN.4.5_1,Pandeglang,"MULTIPOLYGON (((105.21176 -6.76261, 105.21176 ...",2,0,0
8,27270,601,738.0,12,IDN,Indonesia,IDN.4_1,Banten,Province,IDN.4.5_1,Pandeglang,"MULTIPOLYGON (((105.22147 -6.62795, 105.22147 ...",1,0,0
9,27271,601,734.0,12,IDN,Indonesia,IDN.4_1,Banten,Province,IDN.4.5_1,Pandeglang,"MULTIPOLYGON (((105.22156 -6.62804, 105.22156 ...",2,0,0
